In [ ]:
pip install nilearn

In [21]:
# Create images to crop for 'Heatmap object detection'
# These images must be cropped and saved in D:\\Erasmus\\Automation Leon\\Heatmap_object_detection\\reference_imgs\\

# TO DO: Check op voxelsizes inbouwen - print(img.header.get_zooms()) --> (0.451172, 0.451172, 0.625)
# TO DO: Automatische downsamplingfactor i.p.v. hardcoded 1.82 voor pixelsize 512
# TO DO: De lelijke oplossingen voor flippen en roteren netter maken
import numpy as np
from numpy import asarray
import matplotlib
from matplotlib import image
import sys
import nibabel as nib
from nilearn.image import resample_img

DIR_NIFTI    = 'C:\\Users\\Administrator\\OneDrive\\Thesis\\Coding\\Leon\\Heatmap_object_detection\\reference_imgs\\original image\\'
DIR_OUTPUT   = 'C:\\Users\\Administrator\\OneDrive\\Thesis\\Coding\\Leon\\Heatmap_object_detection\\reference_imgs\\to be cropped\\'
NIFTI        = 'ct_test_2001_image.nii.gz'

def create_ref_imgs(filename):
    original_nii = nib.load(filename)
    print('Loading nifti done')
    
    if nib.aff2axcodes(original_nii.affine) != ('R', 'A', 'S'):
        sys.exit('ERROR - Change image orientation to LPI')
    if original_nii.shape[0] != 512 or original_nii.shape[1] != 512:
        sys.exit('ERROR - Image shape should be (512, 512, x)')

    downsampled_nii = resample_img(original_nii, target_affine=np.eye(3)*1.82, interpolation='nearest')
    if downsampled_nii.shape[0] != 128 or downsampled_nii.shape[1] != 128:
        sys.exit('ERROR - Image shape should be (128, 128, x)')
    else:
        print('Downsampling nifti done')
    resampled = np.asanyarray(downsampled_nii.dataobj)
    resampled = np.flip(resampled, 0) # to comply with orientation in viewers such as ITK-Snap or 3D Slicer

    for z in range(resampled.shape[0]):
        outputfile = NIFTI+' S '+str(resampled.shape[0]-z)+'.jpg'
        # to comply with orientation in viewers such as ITK-Snap or 3D Slicer
        img_flipped = np.flipud(resampled[z,:,:])
        img = np.rot90(img_flipped, k=1)
        matplotlib.image.imsave(DIR_OUTPUT+outputfile, img, cmap="gray")
    print('  Creating Sagittal images done')
    
    for y in range(resampled.shape[1]):
        outputfile = NIFTI+' C '+str(y)+'.jpg'
        img = np.rot90(resampled[:,y,:], k=1)
        matplotlib.image.imsave(DIR_OUTPUT+outputfile, img, cmap="gray")
    print('  Creating Coronal images done')
    
    for x in range(resampled.shape[2]):
        outputfile = NIFTI+' A '+str(x)+'.jpg'
        img = np.rot90(resampled[:,:,x], k=1)
        matplotlib.image.imsave(DIR_OUTPUT+outputfile, img, cmap="gray")
    print('  Creating Axial images done')


create_ref_imgs(DIR_NIFTI+NIFTI)

print('All done')

Loading nifti done
Downsampling nifti done
  Creating Sagittal images done
  Creating Coronal images done
  Creating Axial images done
All done
